In [ ]:
import torch
import numpy as np
import pandas as pd
import nibabel as nib
from os import walk

from sklearn.random_projection import SparseRandomProjection

In [ ]:
def loadData(path: str, dtype=torch.float64, flatten=True) -> torch.Tensor:
    trueShape = (128, 128, 49)
    img = nib.load(path)
    #data = img.get_fdata()
    data = np.asarray(img.dataobj)

    if img.shape[:-1] != trueShape:
        print(f"Error: Shape is {img.shape} not {trueShape}")
        return None

    if flatten:
        dataTensor = torch.tensor(np.mean(data, axis=3)).flatten().to(dtype)
    else:
        dataTensor = torch.tensor(np.mean(data, axis=3)).to(dtype)
        
    return dataTensor

In [ ]:
parentPath = '../../ds000201-download/'
scanType = 'func/'
task = '_task-rest_bold.nii.gz'

# Exclude subjects that do not have all scans for both sessions
exclude = ['sub-9016', 'sub-9022', 'sub-9044', 'sub-9066', 'sub-9078', 'sub-9095']


In [ ]:
paths = next(walk(parentPath))[1]
fullData = torch.empty(160, 128*128*49)
count = 0
for sub in paths:
    if sub[:3] != 'sub' or sub in exclude: continue
    for i in range(1,3):
        ses = f'ses-{i}'
        path = parentPath + sub + '/' + ses + '/' + scanType + sub + '_' + ses + task
        print(path)
        fullData[count] = loadData(path) 
        count += 1
torch.save(fullData, 'restData_meanTime_reg.pt')

In [ ]:
dataShape = 128*128*49

X = torch.load('restData_meanTime_reg.pt')


transformer = SparseRandomProjection(random_state=69, eps=.1)
X_new = transformer.fit_transform(X)
xt = torch.tensor(X_new)

torch.save(xt, 'restData_meanTime_spr.pt')
print(X_new.shape)